In [99]:
import pandas as pd
from os import getcwd, path, listdir, chdir

TICKER_PATH = "\data\\tickers\\ticker_data"
columns = ["close", "high", "low", "open", "volume"]
unwanted_cols = ["date", "adjClose", "adjHigh", "adjLow", "adjOpen", "adjVolume", "divCash", "splitFactor"]

In [100]:
def grab_all_tickers():
    #chdir(os.path.abspath(os.path.join(TICKER_PATH)))
    current_dir = getcwd()
    files = [f for f in listdir(current_dir) if path.isfile(path.join(current_dir, f))]
    return files

In [189]:
def moving_averages(df, window=14):
    df_ma = df.rolling(window=14).mean()["open"]
    df_std = df.rolling(window=14).std()["open"]

    df["R.Mean"] = df_ma
    df["R.std"] = df_std
    return df

In [190]:
def stochastic_osc(df, window=14):    
    highest_high = df.rolling(window=14).max()["open"]
    lowest_low = df.rolling(window=14).min()["open"]
    
    df["S0%k"] = (df["open"] - lowest_low) / (highest_high - lowest_low) * 100
    return df

In [191]:
def ema(df, window=14):
    df["EMA"] = df["open"].ewm(span=window).mean()
    return df

In [199]:
def bollinger_bands(df, window=14):
    ma = df["R.Mean"]
    std = df["R.std"]
    open_ = df["open"]
    band1 = 4 * std / ma
    band2 = (open_ - ma + 2 * std) / (4 * std)
    df["BolBand1"] = band1
    df["BolBand2"] = band2
    return df

In [200]:
tickers = grab_all_tickers()
for ticker in tickers:
    temp_df = pd.read_csv(ticker, parse_dates=True, header=0)
    temp_df.drop(labels=unwanted_cols, axis=1, inplace=True)
    final_df = moving_averages(temp_df)
    final_df = stochastic_osc(final_df)
    final_df = ema(final_df)
    final_df = bollinger_bands(final_df)
    final_df.to_csv(f"_Rolling\MA_{ticker}", sep=",", index=False, header=final_df.columns)

In [204]:
first = pd.read_csv(tickers[0], parse_dates=True, header=0)
first.drop(labels=unwanted_cols, axis=1, inplace=True)
first = moving_averages(first)
first = stochastic_osc(first)
first = ema(first)
first = bollinger_bands(first)
first.tail()

,close,high,low,open,volume,R.Mean,R.std,S0%k,EMA,BolBand1,BolBand2
182,107.87,107.87,106.67,106.67,1377,108.068364,1.893058,15.084746,107.478177,0.070069,0.315330
183,106.61,107.12,106.61,107.12,14200,108.164079,1.800048,21.783877,107.430420,0.066567,0.354993
184,106.66,106.84,104.03,105.85,18721,107.988364,1.901895,0.000000,107.219697,0.070448,0.218917
185,105.00,105.00,105.00,105.00,25240,107.631936,1.964470,0.000000,106.923738,0.073007,0.165058
186,104.92,104.92,104.92,104.92,328,107.269793,1.963649,0.000000,106.656573,0.073223,0.200838
